<a href="https://colab.research.google.com/github/MrSalta/Apptentive-Dingus/blob/colab/ApptentiveDingus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# The Apptentive Dingus
To run: Hover over the black code block and click the "Play" button in the top left corner.

In [ ]:
from os import path as osp
import pandas as pd
import inspect
from google.colab import drive
import glob

class StopExecution(Exception):
    def _render_traceback_(self):
        pass

# Mount Google Drive
drive.mount('/content/drive', force_remount=True)

# Define Paths:
apptentive_path = '/content/drive/MyDrive/Apptentive'
key_path= '/content/drive/Shareddrives/Digital GXT/Apptentive/Key.csv'
input_path = '/content/drive/Shareddrives/Digital GXT/Apptentive/InputFiles'

# Select and open key file
dict_key_file = open('/content/drive/Shareddrives/Digital GXT/Apptentive/Key.csv', 'r') 

# Load Key file into dictionary
try:
    dict_from_csv = pd.read_csv(
        dict_key_file,
        header=None,
        index_col=0).squeeze("columns")
    print('Key file successfully loaded.')
  # Handle Key file load error
except FileNotFoundError:
    print('Key file not found. Make sure it\'s in the Google Drive!')
    print(dict_key_file)
    raise StopExecution

# Create dictionary from Key
dict_from_csv = {k.replace(u'\xa0', ' '): v.replace(u'\xa0', ' ')
                 for k, v in dict_from_csv.items()}
print("Dictionary loaded successfully from Key file.")

# Load and read the selected CSVs
f = glob.glob(input_path + "/*.csv")
data = []
print("Files Loaded!")

# Add the 'appname' column to the dataframe
for csv in f:
    frame = pd.read_csv(csv)
    frame['appname'] = osp.basename(csv).split("(")[0]
    data.append(frame)

# DO: Concat the files
df = pd.concat(data)

# Change the question columns to be just the number, for easier reporting
group = df.columns.str.split('.').str[0]
df = df.fillna('').astype(str).groupby(group, axis=1).apply(
    lambda d: d.apply(','.join, axis=1))

# Read through dictionary and replace resultes
df = df.replace(dict_from_csv)

# Remove the "Null" thingy that shows up on the network column
df = df.replace("\<null\>\|", "", regex=True)

# For multi-select columns, concat into one column, separated by commas
# TODO: There needs to be a way for us to declare which "column" is affected by this
# Ok, going to try by asking if there's a multi-select column first
is_multi = input('Are there any multi-select questions in this set? y/n: ')
print(is_multi)
if is_multi.upper() == 'Y':
    print('Multi-Select column indcated. Asking for number.')
    mc = input('Which question number is a multi-select field? ')
    print('Column ' + mc + ' selected.')
    print('mc = ' + mc + '.')

    df[mc] = df[mc].replace(r'\.(?!\s|$)', '', regex=True)
    df[mc] = df[mc].replace(r'\,(?!\w)|(?<!\w)\,', '', regex=True)
    df[mc] = df[mc].str.split(',')
    df = df.explode(mc)

# Get save location
save_location = '/content/drive/Shareddrives/Digital GXT/Apptentive/OutputFile'
save_path = (save_location + '/' + 'GenieNewOutput.csv')

# Actually write the file to the root
df.to_csv(save_path)

print("Load and combine completed. The new file is at " + save_path + ".")


/content
Mounted at /content/drive
Key file successfully loaded.
Dictionary loaded successfully from Key file.
Files Loaded!
Are there any multi-select questions in this set? y/n: y
y
Multi-Select column indcated. Asking for number.
Which question number is a multi-select field? 6
Column 6 selected.
mc = 6.
Load and combine completed. The new file is at /content/drive/Shareddrives/Digital GXT/Apptentive/OutputFile/GenieNewOutput.csv.
